<a href="https://colab.research.google.com/github/riadeb/PI--User-scheduling-in-5G/blob/master/Classifieur_Comparaison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model that predicts the topics from a given sentence. Added Siri voice control commands


---



In [0]:
import pandas as pd
import requests
import numpy as np
import keras
import tensorflow as tf
import tensorflow_hub as hub
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda, Input, Flatten
from keras import Model

url = "https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019/self-dialogs.json"
data = requests.get(url).json()
numofclasses = 11


Using TensorFlow backend.


In [0]:
def transform_array(Y): #transforms Y from class name array to appropriate format
    classes_arr = np.unique(Y)
    classes_dict = dict()
    for i,class_n in enumerate(np.unique(Y)):
        classes_dict[class_n] = i
    for i in range(len(Y)):
        toadd = [0]*numofclasses
        toadd[classes_dict[Y[i]]] = 1
        Y[i] = list(toadd);
    return np.array(Y)

## Adding New Categories

In [0]:
data_raw = pd.read_csv("NLU_Data.csv", sep = ";")

In [0]:
data_raw = data_raw.fillna("Other")

In [0]:
def counting_labels(name, data_raw):
    label_occur_dict = {}
    for i in range(data_raw.shape[0]):
        if data_raw[name][i] not in label_occur_dict.keys():
            label_occur_dict[data_raw[name][i]] = 0
        label_occur_dict[data_raw[name][i]] += 1
    return label_occur_dict

In [0]:
counting_labels("scenario", data_raw)

{'alarm': 582,
 'audio': 412,
 'calendar': 702,
 'cooking': 5,
 'datetime': 714,
 'email': 12,
 'general': 739,
 'iot': 1323,
 'lists': 1,
 'music': 571,
 'news': 844,
 'play': 1163,
 'qa': 22,
 'recommendation': 24,
 'social': 5,
 'takeaway': 407,
 'transport': 10,
 'weather': 1042}

In [0]:
adding_categories = ['music', 'datetime', 'weather', 'news']

In [0]:
def labelization(data_raw):
    dict_ = {}
    for category in adding_categories:
        dict_[category] = []
    for i in range(data_raw.shape[0]):
        if data_raw["scenario"][i] in adding_categories and data_raw["answer_normalised"][i] != "Other":
            dict_[data_raw["scenario"][i]] .append( data_raw["answer_normalised"][i] ) 
    return dict_

In [0]:
command_siri = labelization(data_raw)

In [0]:
command_siri['datetime']

["what's the time in australia",
 'tell me the time in moscow',
 'tell me the time in g. m. t. plus five',
 "tell me today's date",
 'what date is it today',
 "what's the date is currently",
 'whats the day today',
 'thats the date',
 'what is the time',
 'tell me the time',
 'what time is it',
 "what's date today",
 'what day is today',
 'tell me a date',
 'what time is it in las vegas',
 'what time is it in beijing',
 "what's the time",
 'is it five o clock already',
 "what's the time in new york",
 "what's the time in tokyo now",
 'is it night time in peru',
 'it is five hundred and forty five am in liverpool what time is now new york',
 'given it is four hundred and thirty five pm in london what time is it in melbourne',
 'what time is it in moscow now',
 'how many days are left in this year',
 "what day is new year's eve this year",
 'what day is memorial day this year',
 'what day of the week is the twenty second march',
 'twenty ninth march comes on what day',
 'what day of the 

In [0]:
X,Y = [],[]
X_val,Y_val = [],[]
samplestotakefromeachclass = 1000
dd = {'auto': 0,
 'coffee': 0,
 'movie': 0,
 'pizza': 0,
 'restaurant': 0,
 'uber': 0,
 'non-opening':0}
for category in command_siri.keys():
    dd[category] = 0

maxlen = 30
batchsize = 32;
for ut in data:
    sent = ut["utterances"][0]
    class_to_add = ut["instruction_id"].split("-")[0]
    if dd[class_to_add] < samplestotakefromeachclass: 
        X.append(sent["text"])
        Y.append(class_to_add)
        dd[class_to_add] += 1
    else:
        X_val.append(sent["text"])
        Y_val.append(class_to_add)

    class_to_add = 'non-opening'
    if dd[class_to_add] < 2 * samplestotakefromeachclass: 
        X.append(ut["utterances"][np.random.randint(1,len(ut["utterances"]))]["text"])
        Y.append(class_to_add)
        dd[class_to_add] += 1
    else:
        X_val.append(sent["text"])
        Y_val.append(class_to_add)

for category in command_siri.keys():
    for sent in command_siri[category]:
        if dd[category] < samplestotakefromeachclass:
            X.append(sent)
            Y.append(category)
            dd[category] += 1

X = np.array(X)
Y= transform_array(Y)

X_val = np.array(X_val)[:6000]
Y_val = transform_array(Y_val)[:6000]

In [0]:
label = {}
for i in range(len(X)):
    label[X[i]] = Y[i]

In [0]:
list(label.values())[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [0]:
np.random.shuffle(X)

In [0]:
X.shape

(11119,)

In [0]:
Y = []
for i in range(len(X)):
    Y.append(label[X[i]])

In [0]:
Y = np.vstack(Y)

In [0]:
Y.shape

(11119, 11)

## Model Establish and Training

In [0]:
url = "https://tfhub.dev/google/elmo/2"
ELMO = hub.Module(url)

def ELMOO(x):
    return ELMO(tf.squeeze(tf.cast(x,tf.string)),signature="default",as_dict=True)["elmo"]

def build_model_bider():
    input_text = Input(shape=(1,),dtype=tf.string)
    embedding = Lambda(ELMOO, output_shape=(1,1024))(input_text)
    x = Bidirectional(LSTM(units=512, return_sequences=True,recurrent_dropout=0.2, dropout=0.2))(embedding)
    x_rnn = Bidirectional(LSTM(units=512, return_sequences=False,recurrent_dropout=0.2, dropout=0.2))(x)
    out =  Dense(numofclasses, activation="softmax")(x_rnn)
    model = Model(inputs=input_text,outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_model_lstm():
    input_text1 = Input(shape=(None,),dtype=tf.string)
    embeding1 = Lambda(ELMOO, output_shape=(None,1024))(input_text1)
    ltsm = LSTM(128,return_sequences=False)(embeding1)
    out = Dense(numofclasses,activation="softmax")(ltsm)
    model = Model(inputs=input_text1,outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_model_dense():
    def ELMOO1(x):
      return ELMO(tf.squeeze(tf.cast(x,tf.string)),signature="default",as_dict=True)["default"]
    input_text1 = Input(shape=(None,),dtype=tf.string)
    embeding1 = Lambda(ELMOO1)(input_text1)
    dd = Dense(128)(embeding1)
    out = Dense(numofclasses,activation="softmax")(dd)
    model = Model(inputs=input_text1,outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_model_cnn_1d():
    input_text = Input(shape=(None,),dtype=tf.string)
    embedding = Lambda(ELMOO, output_shape=(None,1024))(input_text)
    x = Lambda(lambda x: x[:1024])(embedding)
    cnn1 = Conv1D(128, 5, activation='relu') (embedding)
    cnn2 = Conv1D(128, 5, activation='relu')(cnn1)
    mp = MaxPooling1D(3,1)(cnn2)
    cnn3 = Conv1D(128, 2, activation='relu')(mp)
    cnn4 = Conv1D(128, 2, activation='relu')(cnn3)
    gap = GlobalAveragePooling1D()(cnn4)
    output = Dense(numofclasses, activation="softmax")(gap)
    model = Model(inputs=input_text, output=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
model_lstm = build_model_lstm()
print("model LSTM")
model_lstm.summary()
print(128*"-")
print()
model_bider = build_model_bider()
print("model Bidirection")
model_bider.summary()
print(128*"-")
print()
model_cnn1d = build_model_cnn_1d()
print("model cnn_1d")
model_cnn1d.summary()

print(128*"-")
print()
model_dense= build_model_dense()
print("model dense")
model_dense.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


model LSTM
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
lambda_1 (Lambda)            (None, None, 1024)        0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dense_1 (Dense)              (None, 11)                1419      
Total params: 591,755
Trainable params: 591,755
Non-trainable params: 0
_________________________________________________________________
--------------------------------------------------------------------------------------------------------------------------------

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


model Bidirection
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1, 1024)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 1024)           6295552   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dense_2 (Dense)              (None, 11)                11275     
Total params: 12,602,379
Trainable params: 12,602,379
Non-trainable params: 0
_________________________________________________________________
-----------------------------------------------------------------------------------------------

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


model cnn_1d
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None)              0         
_________________________________________________________________
lambda_3 (Lambda)            (None, None, 1024)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         655488    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         32896     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
# samplestotake = (X.shape[0]//batchsize)*batchsize
# samplestotake_val = (X_val.shape[0]//batchsize)*batchsize
print("LSTM")
model_lstm.fit(X,Y,epochs=4,batch_size=batchsize, shuffle=True, validation_split=0.2)

print("")
print("BIDIR")
model_bider.fit(X,Y,epochs=10,batch_size=batchsize, shuffle=True, validation_split=0.2)


LSTM
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8895 samples, validate on 2224 samples
Epoch 1/4



8895/8895 [==============================] - 73s 8ms/step - loss: 0.7567 - acc: 0.7485 - val_loss: 0.3678 - val_acc: 0.8934
Epoch 2/4
8895/8895 [==============================] - 48s 5ms/step - loss: 0.2519 - acc: 0.9279 - val_loss: 0.2624 - val_acc: 0.9254
Epoch 3/4
8895/8895 [==============================] - 48s 5ms/step - loss: 0.1700 - acc: 0.9494 - val_loss: 0.2732 - val_acc: 0.9290
Epoch 4/4
8895/8895 [==============================] - 48s 5ms/step - loss: 0.1232 - acc: 0.9650 - val_loss: 0.2963 - val_acc: 0.9164

BIDIR
Train on 8895 samples, validate on 2224 samples
Epoch 1/10
8895/8895 [==============================] - 51s 6ms/step - loss: 1.1227 - acc: 0.5798 - val_loss: 0.7158 - val_acc: 0.7469
Epoch 2/10
8895/8895 [==============================] - 46s 5ms/step - loss: 0.6703 - acc: 0.7582 - val_loss: 0.6916 - val_acc: 0.7527
Epoch 3/10
8895/8895 [==============================] - 46s 5ms/step - loss: 0.5723 - acc: 0.7947 - val_loss: 0.6068 - val_acc: 0.7900
Epoch 4/10
889

In [0]:
class_dict = {'auto': 0, 'coffee': 1, 'datetime': 2, 'movie': 3, 'music': 4, 'news': 5, 'non-opening': 6, 'pizza': 7, 'restaurant': 8, 'uber': 9, 'weather': 10}

In [0]:
classes_arr = list(class_dict.keys())

In [0]:
classes_arr

['auto',
 'coffee',
 'datetime',
 'movie',
 'music',
 'news',
 'non-opening',
 'pizza',
 'restaurant',
 'uber',
 'weather']

In [0]:
X = np.array([["Where is the nearest Starbucks ?"],["i need to repair my car"],["I need a ride from home"],
              ["I want to order something to eat"],["Wait a moment"],["I want to go to the airport"], ["This song is really good"],
              ["can you play the song of Mikel Jackson"],["I prefer the red"], ["What is the time"], ["Give me some news today"], 
              ["It's time to leave here"]])
prediction = model_lstm.predict(X)
a = [classes_arr[i] for i in np.argmax(prediction,axis=1)]
print(a)

#prediction = model_bider.predict(X)
#a = [classes_arr[i] for i in np.argmax(prediction,axis=1)]
#print(a)

# prediction = model_cnn1d.predict(X)
# a = [classes_arr[i] for i in np.argmax(prediction,axis=1)]
# print(a)

['coffee', 'auto', 'uber', 'restaurant', 'non-opening', 'uber', 'non-opening', 'music', 'non-opening', 'datetime', 'news', 'uber']


In [0]:
model_bider.save_weights("bider.h5")
model_lstm.save_weights("lstm.h5")

In [0]:
model_lstm = build_model_lstm()
model_lstm.load_weights("lstm.h5")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
text = 'I need a ride' #@param {type:"string"}
X = np.array([[text],[""]])
prediction = model_lstm.predict(X)
a = [classes_arr[i] for i in np.argmax(prediction,axis=1)]
print(a[0])


uber


In [0]:
text = "I want to see a movie tonight#what movie ?#I don't know, what's available ?#I need a ride" #@param {type:"string"}
X = np.array([[t] for t in text.split("#")])
prediction = model_lstm.predict(X)
a = [classes_arr[i] for i in np.argmax(prediction,axis=1)]
print(a)

['movie', 'non-opening', 'non-opening', 'uber']
